<a href="https://colab.research.google.com/github/gastan81/generative-ai/blob/main/0_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot

## 1. Settings

Installation

In [1]:
%%bash
pip install -qqq -U faiss-cpu
#!pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U langchain-huggingface
pip install -qqq -U pypdf
pip install -qqq -U streamlit

In [2]:
%%bash
pip install -qqq -U jupyter
pip install -qqq -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


Libraries

In [2]:
import os
#from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

Colab: token

In [ ]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

Local: token

In [ ]:
# token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

Model

In [5]:
# hf_model = 'mistralai/Mistral-7B-Instruct-v0.3'
# hf_model = 'microsoft/Phi-3.5-mini-instruct'

hf_model = 'Qwen/Qwen2.5-7B-Instruct' # empty outputs

# hf_model = 'mistralai/Mistral-Nemo-Instruct-2407' # long time no response
# hf_model = 'microsoft/phi-4-gguf' # ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='api-inference.huggingface.co', port=443): Read timed out. (read timeout=120)")
# hf_model = 'microsoft/phi-4' # The model microsoft/phi-4 is too large to be loaded automatically (29GB > 10GB).

llm = HuggingFaceEndpoint(repo_id=hf_model)

This will not be needed when everything on github:

https://drive.google.com/file/d/14PqeK1VNokE5PZ-b-hVLqV-h4xnSR7B4/view?usp=sharing Alice in Wonderland

https://drive.google.com/file/d/1mJIwux2e0XvZUDBoEj9rb3OlHlYsTxXb/view?usp=sharing Statistics in Python

https://drive.google.com/file/d/1i5ZhAxtIown7RzH1Sh13y2eqcXKvsMyq/view?usp=sharing Statistics in R

Embedding

In [ ]:
embedding_model = 'Qwen/Qwen2.5-7B-Instruct' # "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "data/cache/"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model
    , cache_folder=embeddings_folder)

2025-01-26 20:48:50.821830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 20:48:51.065224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-26 20:48:51.137572: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 20:48:51.531168: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-26 20:48:54.344031: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Create vector from loaded document

In [17]:
# # The document
# # file_id = '19dUK3V5K8cvu1E5uPzdx_G6uPXMT1zFw' # Google drive ID: CIA world factbook 2018-2019
# # file = 'https://drive.google.com/uc?export=download&id=' + file_id
# file = 'data/The CIA World Factbook 2018-2019.pdf'

In [18]:
# # Read pages
# loader = PyPDFLoader(file)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [8]:
# # Split text
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
# docs = text_splitter.split_documents(pages)

In [ ]:
# # Review the loaded document
# print(f"{docs[10].metadata}\n")
# print(docs[10].page_content)

In [ ]:
# # Create vector
# vector_db = FAISS.from_documents(docs, embeddings)

In [ ]:
# # Save vector
# vector_db.save_local("data/CIA_faiss_index")

Load vector from saved index

In [21]:
vector_db = FAISS.load_local("data/CIA_faiss_index", embeddings, allow_dangerous_deserialization=True)

Retriever

In [22]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

Chat setup

In [31]:
template = """You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation.
Keep your answers short, succinct, informative, and clear, so that the couterpart can learn from you.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:
"""

# chat_history = []

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

doc_retriever = create_history_aware_retriever(
    llm, retriever, prompt
)

doc_chain = create_stuff_documents_chain(llm, prompt)

Chatbot

In [32]:
chain = create_retrieval_chain(
    doc_retriever, doc_chain
)
history = []
# Start the conversation loop
while True:
  user_input = input("\nYou: ")

  # Check for exit condition
  if user_input.lower() == 'end':
      print("Ending the conversation. Goodbye!")
      break

  # Get the response from the conversation chain
  response = chain.invoke({"input":user_input, "chat_history": history, "context": retriever})
  history.extend([{"role": "human", "content": response["input"]},{"role": "assistant", "content":response["answer"]}])
  # Print the chatbot's response
  print(response["answer"])


You: What topic can you talk with me about?


You: Can you tell me the population in Iran?



KeyboardInterrupt: Interrupted by user

Streamlit

In [ ]:
%%writefile data/streamlit_app.py

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS, faiss-cpu
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st

# llm
hf_model = 'mistralai/Mistral-7B-Instruct-v0.3' # 'microsoft/Phi-3.5-mini-instruct'
llm = HuggingFaceEndpoint(repo_id=hf_model)

# embeddings
embedding_model = 'sentence-transformers/all-MiniLM-l6-v2'
embeddings_folder = 'data/cache/'

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model
    , cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local('data/CIA_faiss_index', embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# prompt
template = """You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation.
Keep your answers short, succinct, informative, and clear, so that the couterpart can learn from you.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
])

# bot with memory
@st.cache_resource
def init_bot():
    doc_retriever = create_history_aware_retriever(llm, retriever, prompt)
    doc_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(doc_retriever, doc_chain)

rag_bot = init_bot()


##### streamlit #####

st.title('CIA World Factbook 2018-2019')

# Initialise chat history
# Chat history saves the previous messages to be displayed
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])

# React to user input
if prompt := st.chat_input('Curious minds wanted!'):

    # Display user message in chat message container
    st.chat_message('human').markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({'role': 'human', 'content': prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner('Asking CIA...'):

        # send question to chain to get answer
        answer = rag_bot.invoke({'input': prompt, 'chat_history': st.session_state.messages, 'context': retriever})

        # extract answer from dictionary returned by chain
        response = answer['answer']

        # Display chatbot response in chat message container
        with st.chat_message('assistant'):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({'role': 'assistant', 'content':  response})

Writing data/streamlit_app.py
